<a href="https://colab.research.google.com/github/AnonymousBazinga/IPL_ScorePredictor/blob/main/FinalIPLScore.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install streamlit
!pip install --upgrade gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 49.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 86.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 84.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.9/92.9 kB 12.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.7/302.7 kB 27.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.1/311.1 kB 38.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 671.7 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 407.5/407.5 kB 22.1

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from math import floor
from bs4 import BeautifulSoup
import requests
import re
import streamlit as st
import gradio as gr

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
path = '/content/drive/MyDrive/EXTRAS/triale.csv'
dataset = pd.read_csv(path)
x = dataset.drop(columns = ["Output","MatchID","StadiumID"])
y = dataset["Output"]
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2)

In [8]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(256, input_shape = x_train.shape[1:], activation = 'sigmoid'))
model.add(tf.keras.layers.Dense(64, activation = 'relu'))
model.add(tf.keras.layers.Dense(256, activation = 'softmax'))


In [9]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy', metrics = ['accuracy'])
model.fit(x_train, y_train, epochs=50)

Epoch 1/50
445/445 [==============================] - 2s 3ms/step - loss: 3.8823 - accuracy: 0.0439
Epoch 2/50
445/445 [==============================] - 1s 3ms/step - loss: 3.5949 - accuracy: 0.0682
Epoch 3/50
445/445 [==============================] - 2s 3ms/step - loss: 3.4614 - accuracy: 0.0789
Epoch 4/50
445/445 [==============================] - 1s 3ms/step - loss: 3.4095 - accuracy: 0.0868
Epoch 5/50
445/445 [==============================] - 1s 3ms/step - loss: 3.3461 - accuracy: 0.0930
Epoch 6/50
445/445 [==============================] - 1s 3ms/step - loss: 3.2557 - accuracy: 0.1014
Epoch 7/50
445/445 [==============================] - 1s 3ms/step - loss: 3.1580 - accuracy: 0.1082
Epoch 8/50
445/445 [==============================] - 1s 2ms/step - loss: 3.0843 - accuracy: 0.1146
Epoch 9/50
445/445 [==============================] - 1s 3ms/step - loss: 3.0185 - accuracy: 0.1213
Epoch 10/50
445/445 [==============================] - 2s 4ms/step - loss: 2.9785 - accuracy: 0.1291

In [10]:
model.evaluate(x_test,y_test)

112/112 [==============================] - 0s 2ms/step - loss: 2.4450 - accuracy: 0.2245


[2.445012092590332, 0.22453573346138]

In [11]:
def stadium_avg(stadium_name):
  url = "https://t20-head-to-head.com/statistics-by-ipl-venue/#"
  page = requests.get(url).text
  doc = BeautifulSoup(page,"html.parser")

  temp = 0
  page_text2 = doc.find_all("figure", attrs={'class':'wp-block-table'})
  page_text = page_text2[::2]
  for i in page_text:
    if stadium_name in str(i):
      temp = str(i.table.tbody).split('td')[9]

  stadium_avg = int(temp[1:4])
  return stadium_avg

stadium_avg("Narendra Modi")

169

In [12]:
def fn1(stadium_name, hometeam, runs, overs, wickets):
  avg_score = stadium_avg(stadium_name)
  arr = [[avg_score,hometeam, runs,overs,wickets]]
  t1 = model.predict(arr)
  a = list(t1[0])
  print(a)
  runrate = 0
  for i in a:
    runrate = runrate+(a.index(i)*i)
  out2 = runrate*20
  out = floor(out2)
  return out//10

In [13]:
runs = int(st.number_input("How many runs scored so far?", min_value=0, max_value=256))
overs = float(st.select_slider("How many overs done so far?", options=[str(i)+"."+str(j) for i in range(0,20,1) for j in range(0,6,1)]))
wickets = int(st.select_slider("How many wickets lost far?", options=[i for i in range(0,11,1)]))
hometeam = st.radio("Does the batting team have home court advantage?", ("Yes", "No"))
stadium_name = str(st.text_input("Which stadium is the match being played in?"))
stadium_average = str(stadium_avg(stadium_name))
if hometeam == "Yes":
  hometeam = 1
else:
  hometeam = 0
text = fn1(stadium_name, hometeam, runs, overs, wickets)
st.write("The AI predicted score for this match is", text)

2023-11-13 17:44:19.680 
  command:

    streamlit run /usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py [ARGUMENTS]


1/1 [==============================] - 0s 285ms/step
[7.2311636e-08, 3.482987e-08, 5.959214e-08, 4.4106702e-08, 9.138528e-08, 6.123843e-08, 4.6611337e-08, 4.435752e-08, 4.8441457e-08, 4.3186745e-08, 4.6855167e-08, 4.72743e-08, 3.8793623e-08, 5.9812194e-08, 4.9078658e-08, 5.083806e-08, 5.410347e-08, 5.8362662e-08, 4.9180326e-08, 5.4302486e-08, 6.0398875e-08, 5.3844303e-08, 5.4229464e-08, 7.03876e-08, 6.066946e-08, 6.2972944e-08, 5.046303e-08, 4.2490846e-08, 4.834837e-08, 4.6983498e-08, 4.646505e-08, 6.839346e-08, 5.261531e-08, 7.31334e-08, 3.632242e-08, 3.4698154e-08, 4.6073e-08, 5.5583126e-08, 5.686528e-08, 4.357202e-08, 5.5684467e-08, 7.44252e-08, 0.004844448, 5.3395233e-08, 3.8097347e-08, 4.7136318e-08, 9.2756515e-08, 5.6869293e-08, 6.371689e-08, 5.66114e-08, 4.627953e-08, 5.678762e-08, 4.482838e-08, 3.957004e-08, 4.8941523e-08, 4.896351e-08, 4.4266663e-08, 5.2618375e-08, 2.6124198e-05, 7.282348e-08, 3.6964748e-06, 0.0038987321, 0.0029934067, 4.0039954e-08, 0.014784458, 0.0001841787,

In [16]:
hometeam = gr.Radio([0,1], label="Does the batting team have home court advantage? (0=no,1=yes)")
runs = gr.Number(label="How many runs scored? (e.g. 60, 204)")
overs = gr.Number(label="How many overs finished? (e.g. 10.2, 3, 12.4)")
wickets = gr.Slider(minimum=0,maximum=10,step=1,label="How many wickets are lost?")
out = gr.Number(label="The AI predicted score for this innings is: ")
demo = gr.Interface(fn1,inputs = ['text',hometeam,runs,overs,wickets],outputs=out,theme='glass')
demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://7c8cd01ed289f7c200.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
